**Predict healthcare costs using a regression algorithm.**

You are given a dataset that contains information about different people including their healthcare costs. Use the data to predict healthcare costs based on new data.

In [ ]:
# Import libraries. You may or may not use all of these.
!pip install -q git+https://github.com/tensorflow/docs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

In [ ]:
# Import data
!wget https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv
dataset = pd.read_csv('insurance.csv')
dataset.tail()

In [ ]:
dataset['expenses'].plot(kind='hist', title='expenses')
plt.gca().spines[['top', 'right',]].set_visible(False)

In [ ]:
dataset['children'].plot(kind='hist', title='children')
plt.gca().spines[['top', 'right',]].set_visible(False)

In [ ]:
dataset['bmi'].plot(kind='hist', title='bmi')
plt.gca().spines[['top', 'right',]].set_visible(False)

In [ ]:
dataset['age'].plot(kind='hist', bins=20, title='age')
plt.gca().spines[['top', 'right',]].set_visible(False)

In [ ]:
dataset.groupby('region').size().plot(kind='bar', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

In [ ]:
dataset.groupby('smoker').size().plot(kind='bar', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

In [ ]:
dataset.groupby('sex').size().plot(kind='bar', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

In [ ]:
dataset.info()

In [ ]:
dataset.describe()

In [ ]:
categorical_data = dataset.select_dtypes(include=['object']).columns.tolist()
numerical_data = dataset.select_dtypes(include=['float64', 'int64']).columns.tolist()
print(categorical_data)
print(numerical_data)

In [ ]:
# convert categorical data to numbers
preprocessor = ColumnTransformer([
    ('scale_numeric', MinMaxScaler(), numerical_data),
    ('encode_categorical', OneHotEncoder(), categorical_data)
])

data_numerical_and_scaled = preprocessor.fit_transform(dataset)

In [ ]:
cols = numerical_data + list(preprocessor.named_transformers_['encode_categorical'].get_feature_names_out(categorical_data))
df = pd.DataFrame(data_numerical_and_scaled, columns=cols)
df['expenses'] = dataset['expenses']
df.head()

In [ ]:
df_clean = df.drop(['sex_male', 'smoker_yes'], axis=1)
df_clean.head()

In [ ]:
# 80% of the data as the train_dataset and 20% of the data as the test_dataset
train_dataset, test_dataset = train_test_split(df_clean, test_size=0.2, random_state=0)

# pop off the "expenses" column from these datasets to create new datasets
train_labels = train_dataset.pop('expenses')
test_labels = test_dataset.pop('expenses')

In [ ]:
# create a model and train it with the train_dataset
model = keras.Sequential([
    layers.Input(shape=[len(train_dataset.keys())]),
    layers.Dense(64, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)
])

model.compile(optimizer=tf.keras.optimizers.Adam(0.01),
              loss='mse',
              metrics=['mae', 'mse'])

# use the labels when training the model
history = model.fit(train_dataset, train_labels, epochs=100, validation_split=0.2, verbose=0)

In [ ]:
# RUN THIS CELL TO TEST YOUR MODEL. DO NOT MODIFY CONTENTS.
# Test model by checking how well the model generalizes using the test set.
loss, mae, mse = model.evaluate(test_dataset, test_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))

if mae < 3500:
  print("You passed the challenge. Great job!")
else:
  print("The Mean Abs Error must be less than 3500. Keep trying.")

# Plot predictions.
test_predictions = model.predict(test_dataset).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims,lims)
